In [68]:
import torch
import torch.nn as nn
from fastai.vision.all import *
from fastai.data.all import *
from fastai.distributed import *
import pandas as pd
from pathlib import Path
import time
from modules_th.video_block import * 
from modules_th.inflator import *
from modules_th.triplet_loss import *

In [2]:
#export
import torch
import torch.nn as nn
from fastai.vision.all import *
from IPython.display import display, clear_output
from fastai.data.all import *
from fastai.distributed import *
from fastscript import *
import pandas as pd
from pathlib import Path
import time
from video_block import * 
from inflator import *
from triplet_loss import *

ModuleNotFoundError: No module named 'video_block'

In [3]:
#exporti
def read_data(eugenio):
    path_data = '/mnt/data/eugeniomarinelli/videodata/df.csv' if eugenio else '/mnt/data/adrianlopez/Videos/Charades/df.csv'
    df = pd.read_csv(path_data, index_col=0).dropna()
    return df 

def get_vid_path(df:pd.Series): return df['paths']
def get_lbls(df:pd.Series): return df['lbl']

@call_parse
def main(gpu    :Param("GPU to run on", int)=None,
         eugenio:Param("Variable to control whether is Eugenio or me that's calling the trianing", bool)=False,
         n_lbl  :Param("# of different labels per batch", int)=4,
         n_el   :Param("# of elements per label", int)=8,
         l      :Param("Number of frames to ersize video to", int)=20
        ):
    
    gpu = setup_distrib(gpu)
    
    df = rank0_first(lambda: read_data(eugenio))

    tfms = [[get_vid_path, Video.create],
            [get_lbls, Categorize()]]
    splits = RandomSplitter(seed=42)(df)
    dsets = Datasets(df, tfms, splits=splits)

    dls = dsets.dataloaders(n_lbl=n_lbl, n_el=n_el, 
                            dl_type=UniformizedDataLoader,
                            after_item=[ResizeTime(l=l), Resize(224), ToTensor()],
                            after_batch=[IntToFloatTensor(), Normalize.from_stats(*imagenet_stats)])

    head, body = inflate(create_head(1024, len(dls.vocab), lin_ftrs=[256])), inflate(create_body(resnet34, cut=-2)) 
    learn = Learner(dls, 
                    TLModel(body,head),
                    loss_func=MixedLoss(alpha=1.0, margin=.2),
                    splitter=my_splitter,
                    metrics=tl_accuracy)
    
    torch.cuda.empty_cache()
    with learn.distrib_ctx(gpu):
        learn.fine_tune(8)
        learn.save('trained_models/Inflated1')

In [7]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_triplet_loss.ipynb.
Converted 02_inflator.ipynb.
Converted 03_video_block.ipynb.
Converted 04_datasets.ipynb.
Converted 05_distributed_inflated_NN.ipynb.
Converted 06_first_inflated_NN.ipynb.
Converted index.ipynb.


In [6]:
# default_exp first_inflated_NN